In [1]:
import numpy as np
from pathlib import Path
import re
from tqdm import tqdm
from skimage import io
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
#from pystackreg import StackReg
#from plantcv import plantcv as pcv
#from VascGraph.Skeletonize import Skeleton

#from VascGraph.GraphIO import ReadStackMat
#from VascGraph.GraphLab import StackPlot
#from VascGraph.Tools.VisTools import visG
#from VascGraph.Tools.CalcTools import fixG
#from VascGraph.GraphIO import WritePajek

In [2]:
#!pip install pystackreg

In [3]:
def remove_small_comps_3d(image, thresh = 1000):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

In [4]:
directory = Path('james_preds')
files  = directory.glob('*_mean.npy')
files = sorted([x.as_posix() for x in files])

In [5]:
#files

In [6]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[:]):
    mean = np.load(file)
    std = np.load(re.sub('mean','std',file))
    seg = np.zeros(mean.shape[1:])
    seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
    seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
    seg = seg.astype('int8')
    seg = (seg==1)*1
    seg = fill_holes(seg)
    np.save(re.sub('mean','seg',file),seg)
    savemat(re.sub('mean.npy','seg.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
    tif.imwrite(re.sub('mean.npy','seg.tif',file),seg)

  6%|▋         | 6/94 [02:17<33:38, 22.94s/it]


KeyboardInterrupt: 

In [8]:
files_seg_0001 = directory.glob('*_0001_seg.npy')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
files_seg_0001

['james_preds/XYZres006_0001_seg.npy',
 'james_preds/XYZres007_0001_seg.npy',
 'james_preds/XYZres008_0001_seg.npy',
 'james_preds/XYZres009_0001_seg.npy',
 'james_preds/XYZres010_0001_seg.npy',
 'james_preds/XYZres011_0001_seg.npy',
 'james_preds/XYZres012_0001_seg.npy',
 'james_preds/XYZres013_0001_seg.npy',
 'james_preds/XYZres014_0001_seg.npy',
 'james_preds/XYZres015_0001_seg.npy',
 'james_preds/XYZres016_0001_seg.npy',
 'james_preds/XYZres017_0001_seg.npy',
 'james_preds/XYZres018_0001_seg.npy',
 'james_preds/XYZres019_0001_seg.npy',
 'james_preds/XYZres020_0001_seg.npy',
 'james_preds/XYZres021_0001_seg.npy',
 'james_preds/XYZres022_0001_seg.npy',
 'james_preds/XYZres023_0001_seg.npy',
 'james_preds/XYZres024_0001_seg.npy',
 'james_preds/XYZres025_0001_seg.npy',
 'james_preds/XYZres026_0001_seg.npy',
 'james_preds/XYZres027_0001_seg.npy',
 'james_preds/XYZres028_0001_seg.npy',
 'james_preds/XYZres029_0001_seg.npy',
 'james_preds/XYZres030_0001_seg.npy',
 'james_preds/XYZres031_0

In [9]:
for file_0001 in tqdm(files_seg_0001):
    file = re.sub('_0001','',file_0001)
    skel_file = re.sub('seg.npy','skel.mat',file)
    seg = np.load(file)
    skel = loadmat(skel_file)
    skel = loadmat(skel_file)['FilteredImage']
    skel = skeletonize_3d(skel)
    skel = remove_small_comps_3d(skel,10)
    print(np.sum(skel))
    dst_tsfm = distance_transform_edt(seg)*0.99436
    skel_dst = skel*dst_tsfm
    
    skel_file_0001 = re.sub('seg.npy','skel.mat',file_0001)
    seg_0001 = np.load(file_0001)
    skel_0001 = loadmat(skel_file_0001)
    skel_0001 = loadmat(skel_file_0001)['FilteredImage']
    skel_0001 = skeletonize_3d(skel_0001)
    skel_0001 = remove_small_comps_3d(skel_0001,10)
    dst_tsfm_0001 = distance_transform_edt(seg_0001)*0.99436
    skel_dst_0001 = skel_0001*dst_tsfm_0001
    
    np.save(re.sub('seg','dst_skel',file),skel_dst)
    io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
    graph = sknw.build_sknw(skel_dst, multi=False)
    
    np.save(re.sub('seg','dst_skel',file_0001),skel_dst_0001)
    io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
    graph_0001 = sknw.build_sknw(skel_dst_0001, multi=False)
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    nx.write_gpickle(graph,re.sub('_seg.npy','.pickle',file))
    nx.write_gpickle(graph_0001,re.sub('_seg.npy','.pickle',file_0001))
    #graph = skan.summarize(skan.Skeleton(skel_dst))
    #raph.to_csv(re.sub('seg.npy','graph.csv',file))

  0%|          | 0/47 [00:00<?, ?it/s]

23427


/tmp/ipykernel_260671/49184904.py:23: UserWarning: james_preds/XYZres006_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file),skel_dst)
/tmp/ipykernel_260671/49184904.py:27: UserWarning: james_preds/XYZres006_0001_dst_skel.tif is a low contrast image
  io.imsave(re.sub('seg.npy','dst_skel.tif',file_0001),skel_dst_0001)
  2%|▏         | 1/47 [00:41<31:31, 41.11s/it]


KeyboardInterrupt: 

In [22]:
skel_dst_0001.shape

(507, 507, 252)

In [31]:
re.sub('_seg.npy','.pickle',file)

'james_preds/XYZres006_0001.pickle'

In [ ]:
skel_file = re.sub('seg.npy','skel.mat',file)

In [27]:
graph[0][39]

{'pts': array([[  3, 419,  33],
        [  4, 419,  33],
        [  5, 419,  33],
        [  6, 419,  32],
        [  7, 419,  31],
        [  8, 419,  31],
        [  9, 419,  31],
        [ 10, 419,  30],
        [ 11, 419,  29],
        [ 12, 419,  28],
        [ 13, 419,  27],
        [ 14, 419,  27],
        [ 15, 420,  26],
        [ 16, 420,  25],
        [ 17, 420,  25],
        [ 18, 420,  25],
        [ 19, 420,  25],
        [ 20, 420,  25],
        [ 21, 420,  24],
        [ 22, 420,  23],
        [ 23, 420,  22],
        [ 24, 420,  21],
        [ 25, 420,  20],
        [ 26, 420,  19],
        [ 26, 420,  18],
        [ 26, 420,  17],
        [ 27, 421,  16],
        [ 28, 421,  15],
        [ 29, 421,  14],
        [ 30, 421,  14],
        [ 31, 421,  13],
        [ 31, 421,  12],
        [ 32, 421,  11],
        [ 33, 421,  10],
        [ 34, 421,  10],
        [ 35, 421,   9],
        [ 36, 421,   9],
        [ 37, 421,   8],
        [ 38, 421,   9],
        [ 39, 421,

In [ ]:
re.sub('seg.npy','graph.csv',file_0001)

In [ ]:
files[22:24]